In [4]:
!pip install qiskit
!pip install qiskit-aer
!pip install qiskit-algorithms

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 39.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.5/108.5 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 69.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.5/310.5 kB 2.6 MB/s eta 0:00:00


**Находим разложение заданной матрицы в комбинацию строк Паули**

In [7]:
from sympy import solve, Matrix
from sympy.physics.quantum import TensorProduct

M = Matrix([[-1.06365335, 0, 0, 0.1809312],
            [0, -1.83696799, 0.1809312, 0],
            [0, 0.1809312, -0.24521829, 0],
            [0.1809312, 0, 0, -1.06365335]])

I = Matrix([[1, 0],
            [0, 1]])
X = Matrix([[0, 1],
            [1, 0]])
Z = Matrix([[1, 0],
            [0, -1]])

II = TensorProduct(I, I)
IZ = TensorProduct(I, Z)
ZI = TensorProduct(Z, I)
ZZ = TensorProduct(Z, Z)
XX = TensorProduct(X, X)

x = Matrix(["x_II", "x_IZ", "x_ZI", "x_ZZ", "x_XX"])
coeffs = solve(x[0] * II + x[1] * IZ + x[2] * ZI + x[3] * ZZ + x[4] * XX - M, x, dict=True)[0]

coeffs = [(str(coeff)[2:], value) for coeff, value in coeffs.items()]
coeffs

[('II', -1.05237324500000),
 ('IZ', 0.397937425000000),
 ('XX', 0.180931200000000),
 ('ZI', -0.397937425000000),
 ('ZZ', -0.0112801050000000)]

**Инициализация**

In [9]:
from qiskit.quantum_info import SparsePauliOp

M = SparsePauliOp.from_list(coeffs)

print(f"Number of qubits: {M.num_qubits}")

Number of qubits: 2


In [10]:
# define ansatz and optimizer
from qiskit.circuit.library import TwoLocal
from qiskit_algorithms.optimizers import SPSA

iterations = 100
ansatz = TwoLocal(rotation_blocks="ry", entanglement_blocks="cx")
spsa = SPSA(maxiter=iterations)

In [13]:
# define Aer Estimator for noiseless statevector simulation
from qiskit_algorithms.utils import algorithm_globals
from qiskit_aer.primitives import Estimator as AerEstimator

seed = 170
algorithm_globals.random_seed = seed

noiseless_estimator = AerEstimator(
    run_options={"seed": seed, "shots": 1024},
    transpile_options={"seed_transpiler": seed},
)

**Запуск VQE и получение результата**

In [15]:
# instantiate and run VQE
from qiskit_algorithms import VQE

vqe = VQE(noiseless_estimator, ansatz, optimizer=spsa)
result = vqe.compute_minimum_eigenvalue(operator=M)

print(f"VQE on Aer qasm simulator (no noise): {result.eigenvalue.real:.5f}")

VQE on Aer qasm simulator (no noise): -1.85160


**Среди частиц, представленных в таблице, наиболее близкой энергией к полученной с помощью алгоритма VQE обладает $H_2$**